In [2]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
from torch.utils.data import dataset
from PIL import Image, ExifTags
import colorsys
import cv2
from pycocotools.coco import COCO
import keyboard

In [3]:
data_path = '/opt/ml/input/data'
file_name = 'train.json'
check_file_name = 'check_list.txt'
with open(os.path.join(data_path, file_name), 'r') as f:
    dataset = json.loads(f.read())

In [4]:
dataset.keys() # dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

dataset['info'] 
# {'year': 2021,
#  'version': '1.0',
#  'description': 'Recycle Trash',
#  'contributor': 'Upstage',
#  'url': None,
#  'date_created': '2021-02-02 01:10:00'

dataset['licenses']
# [{'id': 0,
#   'name': 'CC BY 4.0',
#   'url': 'https://creativecommons.org/licenses/by/4.0/deed.ast'}]

dataset['images'][0]
# {'license': 0,
#  'url': None,
#  'file_name': 'batch_01_vt/0003.jpg',
#  'height': 512,
#  'width': 512,
#  'date_captured': None,
#  'id': 0}

dataset['categories']
# [{'id': 0, 'name': 'UNKNOWN', 'supercategory': 'UNKNOWN'},
#  {'id': 1, 'name': 'General trash', 'supercategory': 'General trash'},
#  {'id': 2, 'name': 'Paper', 'supercategory': 'Paper'},
#  {'id': 3, 'name': 'Paper pack', 'supercategory': 'Paper pack'},
#  {'id': 4, 'name': 'Metal', 'supercategory': 'Metal'},
#  {'id': 5, 'name': 'Glass', 'supercategory': 'Glass'},
#  {'id': 6, 'name': 'Plastic', 'supercategory': 'Plastic'},
#  {'id': 7, 'name': 'Styrofoam', 'supercategory': 'Styrofoam'},
#  {'id': 8, 'name': 'Plastic bag', 'supercategory': 'Plastic bag'},
#  {'id': 9, 'name': 'Battery', 'supercategory': 'Battery'},
#  {'id': 10, 'name': 'Clothing', 'supercategory': 'Clothing'}]

dataset['annotations'][1]
# {'id': 0,
#  'image_id': 0,      # 이미지 같은거 안에 여러개 포함가능. 즉 annotation id > image_id
#                      # 이미지 오름차순으로 저장됨.
#  'category_id': 8,
#  'segmentation': [[184,
#    455,
#    183,
#    455,
#    182,
#    152,
#    188,
#    151,
#    ...]],
#  'area': 79999.0,
#  'bbox': [128.2, 107.0, 381.3, 348.8],
#  'iscrowd': 0}

cnt_img = len(dataset['images'])
cnt_annotation = len(dataset['annotations'])
print(f'annotation 수 {cnt_annotation}, image 수 {cnt_img}, 나누기 {cnt_annotation/cnt_img}') # 21116 > 이미지  평균 8개 알수있음

annotation 수 21116, image 수 2617, 나누기 8.068781047000382


In [116]:
def checkData(start_id = 0, check_file_path = 'check_list.txt'):
#     start_id에는 보고싶은 id값을 입력해주시면됩니다. 이전에 멈춘 부분부터 보기 쉽게 X를 이용한 종료시 id값을 출력하고 종료합니다.
#     노트북에서 사진을 지웠다가 다시 출력하는 기능을 추가하고싶어 찾았지만 찾지 못했습니다. 해당기능이 있으면 저장용량도 작아지고 셀삭제를 반복하지 않아도 되서 편할것 같습니다.
#     check_file_path에는 이상하거나 추가하고싶은 데이터 목록을 저장하는 경로를 지정해주세요.
#     이후 showCheckedData 에서 해당 경로의 파일을 불러와 보여줍니다.    
#     코멘트는 이왕이면 영어로?
    idx = start_id
    anno_idx = 0
    while dataset['annotations'][anno_idx]['image_id'] < dataset['images'][idx]['id']:
        anno_idx += 1

    check_file = open(check_file_path,'a')


    while True :
        plt.figure(figsize=(10,10))#inch 단위 w,h
        img_name = dataset['images'][idx]['file_name']
        img_path = os.path.join(data_path, img_name)
        img_id = dataset['images'][idx]['id']
        
        img = Image.open(img_path)
        plt.title(f'img ID : {img_id}, file path : {img_path}')
        plt.imshow(img)

        #draw annotations
        cls_color = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99] # 11 class color
        # Plastic bac = 보라색.
        cls_cnt = [0]* 11
        anno_img = plt.gca()
        anno =dataset['annotations'][anno_idx]
        while anno['image_id'] == img_id:
        #     print(anno['id'])
            color = colorsys.hsv_to_rgb(cls_color[anno['category_id']],1,1)
            cls_cnt[anno['category_id']] += 1
            x,y,w,h = anno['bbox']
            anno_img.add_patch(patches.Rectangle((x,y),w,h, linewidth = 3,fill=False, edgecolor=color, alpha = 0.5))
            anno_idx += 1
            anno =dataset['annotations'][anno_idx]
        
        category_names = ['UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 
                  'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
        cnt_str = ''
        for i in range(0, 11):
            cnt_str += category_names[i]+': '+str(cls_cnt[i])+',  ' 
        plt.xlabel(cnt_str)
        plt.show()
        
        print('press N, C, X, (아무것도 입력 없어도 다음)')
        
        x = input().upper()
        if x == '':
            idx +=1
            continue
        while x not in ['N','X','C']:
            x = input().upper()
       
        if x == 'X':
            check_file.close()
            print(f'last img_id = {img_id}')
            break
        elif x == 'N':
            idx +=1
            continue
        else:
            print('check comment')
            chk_comment = input()
            while chk_comment == '':
                print('check comment (ENG)')
                chk_comment = input()
            check_file.write(str(img_id)+'\t'+img_path+'\t'+chk_comment+'\n')
            idx +=1

In [117]:
def showCheckedData(check_file_path = 'check_list.txt'):
    check_file = open(check_file_path,'r')
    
    check_list = []
    while True:
        line = check_file.readline()
        if not line: break
        check_list.append(line.split('\t'))
        
    check_list.sort(key = lambda x : int(x[0]))
    
    idx = 0
    anno_idx = 0
    while True:
        #idx 증가 -> annotation 찾아야됨.
        img_id, img_path, comment = check_list[idx]
        img_id = int(img_id)
        while dataset['annotations'][anno_idx]['image_id'] < img_id:
            anno_idx += 1
        anno = dataset['annotations'][anno_idx]

        
        plt.figure(figsize=(10,10))#inch 단위 w,h
        img = Image.open(img_path)
        plt.title(f'img ID : {img_id}, file path : {img_path}')
        plt.imshow(img)

        #draw annotations
        cls_color = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99] # 11 class color
        cls_cnt = [0]* 11
        anno_img = plt.gca()
        anno =dataset['annotations'][anno_idx]
        while anno['image_id'] == img_id:
        #     print(anno['id'])
            color = colorsys.hsv_to_rgb(cls_color[anno['category_id']],1,1)
            cls_cnt[anno['category_id']] += 1
            x,y,w,h = anno['bbox']
            anno_img.add_patch(patches.Rectangle((x,y),w,h, linewidth = 3,fill=False, edgecolor=color, alpha = 0.7))
            anno_idx += 1
            anno =dataset['annotations'][anno_idx]
        
        category_names = ['UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 
                  'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
        cnt_str = ''
        for i in range(0, 11):
            cnt_str += category_names[i]+': '+str(cls_cnt[i])+',  ' 
        plt.xlabel(comment+cnt_str)
        plt.show()
        
        print('press X exit')
        
        x = input().upper()
        if x != 'X':
            idx +=1
            continue
        else:
            break